In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05269455909729004
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 11.495813369064031
200000 19.182337095741335
300000 24.56793084709441
400000 28.63776626646148
500000 29.297921245693196
600000 30.17653943769715
700000 31.908741339809435
800000 38.66972871900223
900000 40.92573505538824
1000000 47.131330535478284
1100000 49.556009864899245
1200000 35.78253507704743
1300000 26.27809912539719
1400000 47.91285833864996
1500000 48.266694160770484
1600000 48.58284856271273
1700000 48.68079239017939
1800000 51.34373800095778
1900000 50.81635077208527
2000000 48.760668081403495
2100000 48.325508966513084
2200000 48.275333581787834
2300000 49.37689851681683
2400000 40.00936185043887
2500000 20.211912629234728
2600000 47.60228014652477
2700000 47.911084556113444
2800000 47.44492618906476
2900000 46.59664266137799
3000000 46.22761

29800000 46.698610099422915
29900000 46.38552382270345
30000000 43.8227519462987
30100000 22.803788641958853
30200000 36.67450829059767
30300000 45.11941500447545
30400000 45.79814351377905
30500000 46.36138294091487
30600000 46.43391639834459
30700000 46.372593239172566
30800000 46.62263909636718
30900000 47.389789172658475
31000000 47.62878049056569
31100000 23.50536883192983
31200000 44.73934738587625
31300000 49.334998765253836
31400000 52.36379120458174
31500000 43.092278453094444
31600000 26.29761943600004
31700000 50.37885789861178
31800000 50.97055804859359
31900000 50.58161112689925
32000000 50.153673447853194
32100000 51.14968573567583
32200000 49.37478749021919
32300000 48.22143280873481
32400000 46.69465081629783
32500000 25.23139155060562
32600000 34.40803842896305
32700000 45.599117204350556
32800000 48.65512573747494
32900000 49.65647914813084
33000000 49.044373074203314
33100000 48.5580381453116
33200000 50.467711194758685
33300000 51.981487886870404
33400000 50.3609859

59900000 46.61802506351492
60000000 45.16501742444221
60100000 44.84384587414132
60200000 46.62781552235744
60300000 47.94489955262573
60400000 47.77659036933557
60500000 48.681251992782606
60600000 36.837469801508504
60700000 27.440574427989795
60800000 46.64696849798367
60900000 45.775263448086
61000000 45.618954649831394
61100000 44.057306953987506
61200000 43.65165467963248
61300000 41.091794402254926
61400000 13.580462073459483
61500000 41.30017782947214
61600000 43.062043038553625
61700000 43.91926796445648
61800000 42.476551898713616
61900000 45.67040117714225
62000000 44.317636010958516
62100000 38.252847066001365
62200000 28.677973199666553
62300000 48.52757078247468
62400000 48.184582157165224
62500000 47.49671030663407
62600000 47.4424622272938
62700000 46.37359606322023
62800000 46.28399698131203
62900000 46.96046118486488
63000000 43.857809511321264
63100000 18.45195880802699
63200000 43.51309580975977
63300000 43.515664827721146
63400000 41.09633699656391
63500000 40.5430

90000000 41.40284573789205
90100000 38.925655219633676
90200000 38.55766562939275
90300000 12.625983275166863
90400000 40.24272479297405
90500000 40.46138453043614
90600000 38.36255739775818
90700000 39.17583700821355
90800000 20.782138188495175
90900000 36.370498191704144
91000000 43.47424241109914
91100000 41.265967967832346
91200000 44.40410362583892
91300000 44.06126310682562
91400000 42.48236767719995
91500000 42.66910949054177
91600000 42.616744325418956
91700000 42.15698246246857
91800000 42.264627462014
91900000 40.52454780130486
92000000 40.637098497751715
92100000 33.74329540121115
92200000 20.482325220378243
92300000 38.471141745653775
92400000 36.80712493848098
92500000 39.24011813284377
92600000 38.99291851069294
92700000 9.535855427232566
92800000 40.7140209190315
92900000 37.72424788073063
93000000 38.498637240393684
93100000 37.63550862044681
93200000 17.28307271976487
93300000 38.408153019284185
93400000 40.94057942522959
93500000 40.62901198309432
93600000 41.14950511

119400000 34.98453059111301
119500000 18.31861852316517
119600000 36.265962930141825
119700000 37.23399995588386
119800000 35.921453793148544
119900000 38.56419058086356
120000000 40.11368924657572
120100000 41.053338779437354
120200000 41.53981691039432
120300000 41.54442355131773
120400000 39.34995026494166
120500000 39.94035970281075
120600000 39.046197082227195
120700000 37.816961343516105
120800000 36.543711249697594
120900000 37.207221268251445
121000000 36.29591058774929
121100000 36.30477307765484
121200000 27.39581520546488
121300000 21.427571289088373
121400000 34.62755934670209
121500000 27.30192063354493
121600000 19.048560948904843
121700000 34.2111652810942
121800000 30.932675555865384
121900000 20.593247754324953
122000000 36.01671010684443
122100000 35.91090843789753
122200000 35.55066122317141
122300000 36.74788948546073
122400000 36.84812651887893
122500000 39.50990450382431
122600000 39.48767596585251
122700000 39.00032892048697
122800000 40.26650630173599
122900000 

...

493900000 3.903704353556242
494000000 8.602445779669134
494100000 7.683822785508534
494200000 7.438248758359942
494300000 7.770187770872853
494400000 7.025890544424412
494500000 5.259087773222248
494600000 6.427138241409248
494700000 8.18077109058482
494800000 7.86432062749093
494900000 8.089293928732047
495000000 8.187681370275264
495100000 3.6401064328933175
495200000 8.412762598501818
495300000 8.07462130791187
495400000 7.695794729760565
495500000 7.706824642462546
495600000 6.560456301846263
495700000 6.64905353183886
495800000 5.91477915151325
495900000 8.093121986545848
496000000 7.772803290269389
496100000 8.093019435488063
496200000 7.870926662051628
496300000 3.8871323763423384
496400000 7.759124143681043
496500000 8.156872107322474
496600000 7.5538809260792155
496700000 7.514008678541375
496800000 5.7189462457237
496900000 7.070261785905322
497000000 5.857235037358394
497100000 8.333821449660178
497200000 7.639292838646562
497300000 7.881314278647233
497400000 7.689614225115

523200000 5.672398986900585
523300000 5.741449955346022
523400000 6.002636008041668
523500000 5.913076716309494
523600000 5.7737083877652084
523700000 5.688261988000212
523800000 5.968218365549213
523900000 4.3276146835589415
524000000 2.8000003499722323
524100000 3.4509440304846852
524200000 5.4135269564148105
524300000 5.744056595393335
524400000 5.578821229370963
524500000 5.770068684692351
524600000 5.623293458011121
524700000 5.570413334233989
524800000 5.425155916594998
524900000 5.196709198484455
525000000 5.760291140418732
525100000 4.204320752958288
525200000 2.875980443960702
525300000 3.496876653324285
525400000 5.16905221428346
525500000 5.157743152421885
525600000 5.315036775065808
525700000 5.358015746830677
525800000 5.728175943186156
525900000 5.86622164700567
526000000 5.705849547201407
526100000 5.7171486099000814
526200000 5.564117310543959
526300000 4.378315280250687
526400000 1.9487159090325035
526500000 4.182613505521801
526600000 5.7619322943077025
526700000 5.31

552200000 4.827051410966885
552300000 5.006934302760814
552400000 4.474569834007259
552500000 3.3011722348068835
552600000 3.797456474346024
552700000 4.1426955173404565
552800000 3.71780531160814
552900000 2.7486041814740547
553000000 4.148056010752864
553100000 4.352051071232361
553200000 3.125277182257163
553300000 4.4573599126352645
553400000 4.636058282613526
553500000 4.766527863164159
553600000 4.3560021402183775
553700000 2.882487073515817
553800000 4.19631831994328
553900000 4.086040629272869
554000000 3.7395047149284726
554100000 2.6225420703436337
554200000 4.036999918615652
554300000 4.125635801251682
554400000 3.195780683871505
554500000 4.441651479575862
554600000 4.545044308084259
554700000 4.597886314649302
554800000 4.573809117737211
554900000 2.533601401065338
555000000 4.398535852611143
555100000 4.125592106423302
555200000 3.6790090799649637
555300000 2.4422855343209866
555400000 4.078724979754362
555500000 4.122608828211603
555600000 2.9956691020235566
555700000 4.

579800000 2.182544946565588
579900000 2.105096937565754
580000000 2.6508366506727854
580100000 2.565444324954635
580200000 2.1841489261471927
580300000 1.9708751161598754
580400000 2.085591061568131
580500000 2.0976565108920924
580600000 2.7842139585862737
580700000 2.755887262429589
580800000 1.8679346897200644
580900000 1.9939048087219404
581000000 1.976009757654065
581100000 1.8676302844206842
581200000 2.8561534783523665
581300000 2.7242667792838895
581400000 1.791719092012798
581500000 1.9774065778838732
581600000 1.8327975798121354
581700000 1.9000263444809542
581800000 2.757384478972183
581900000 2.59558007703529
582000000 1.7635130553599419
582100000 1.8997752263033316
582200000 1.9119020821092467
582300000 1.8401155114134762
582400000 2.715603437979293
582500000 2.7553686205878796
582600000 1.7912191489317744
582700000 1.970473246113917
582800000 1.8353394346974017
582900000 1.8588985685324715
583000000 2.592465168327045
583100000 2.4058763257943965
583200000 1.718382497427193

608300000 1.283019788400483
608400000 1.0141107733733468
608500000 1.2163595246920593
608600000 0.675260804136918
608700000 1.037231655922034
608800000 0.9787596972160459
608900000 1.0578050540674746
609000000 0.8667958621912566
609100000 1.274503943478771
609200000 0.8011171393537919
609300000 1.233780179700617
609400000 1.1392679121480631
609500000 1.0332040397861082
609600000 0.9085878147528271
609700000 1.1770703115377732
609800000 0.7232822376833319
609900000 0.9450081904877735
610000000 1.0373432609906144
610100000 0.5190270215970239
610200000 0.5769727777486648
610300000 1.0350278989023882
610400000 0.7941774647367636
610500000 1.009924384769286
610600000 1.2131444044752266
610700000 0.7302664789895952
610800000 0.8978076545342234
610900000 0.9652226406240371
611000000 0.8022776843538504
611100000 1.0071868239905906
611200000 1.2029218412127072
611300000 0.4319209408971945
611400000 0.7754912444150959
611500000 0.6802040055338898
611600000 0.6543131569105982
611700000 0.98452769